# Analysis of Fake News

## Introduction

This project focuses on analyzing fake news and builds a model to predict fake news in the future.


This is my Capstone Project-1 for my coursework at Springboard.

Credit to Clément Bisaillon for providing the dataset. https://www.kaggle.com/clmentbisaillon/fake-and-real-news-dataset

## Step 1: Uploading the Datasets

In [1]:
#import required packages

import pickle
import os
import numpy as np
import pandas as pd

#Importing the real dataset
cols = ['title','text','subject']
os.chdir('/Users/purpleblack/Documents/Learning/Springboard/Capstone Project 1/The Project')
real = pd.read_csv('True.csv', usecols = cols)
real['Real/Fake'] = 'REAL'
real.head()
   

,title,text,subject,Real/Fake
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,REAL
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,REAL
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,REAL
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,REAL
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,REAL


In [2]:
#Importing the fake dataset
fake = pd.read_csv('Fake.csv', usecols = cols)
fake['Real/Fake'] = 'FAKE'
fake.head()

#I've assigned an entire columns to be filled with FAKE or REAL as both datasets were homogeneous.

,title,text,subject,Real/Fake
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,FAKE
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,FAKE
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,FAKE
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,FAKE
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,FAKE


## Step 2: Merging the Datasets

In [3]:
dataset = real.copy()
dataset = dataset.append(fake, ignore_index = True)
dataset.tail()

,title,text,subject,Real/Fake
44893,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,FAKE
44894,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,FAKE
44895,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,FAKE
44896,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,FAKE
44897,10 U.S. Navy Sailors Held by Iranian Military ...,21st Century Wire says As 21WIRE predicted in ...,Middle-east,FAKE


## Step 3: Saving the Data for Later

In [4]:
!mkdir news_dataset
with open('dataset.txt','wb') as file:
    pickle.dump(dataset,file)

mkdir: news_dataset: File exists


## Step 4: Cleaning the Data


I will be doing the following on the text data:

   * Converting all characters to lower case
   * Removing punctuation
   * Tokenize text
   * Remove stop words
   * Remove words with numbers

In [5]:
#Making the first pass of cleaning.

import re
import string

def clean1(text):
    '''Converts the text to lower case, removes text in brackets and removes punctuation '''
    text = text.lower()
    text = re.sub('\(.*\)','',text) #Removes text in brackets
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)  #Removes punctuation
    text = re.sub('\w*\d\w*', '', text) #Removes digits in words
    return text

round_1 = lambda x: clean1(x)

In [6]:
#Applying the function to the title column
title_clean = pd.DataFrame(dataset.title.apply(round_1))
title_clean

,title
0,as us budget fight looms republicans flip thei...
1,us military to accept transgender recruits on ...
2,senior us republican senator let mr mueller do...
3,fbi russia probe helped by australian diplomat...
4,trump wants postal service to charge much more...
...,...
44893,mcpain john mccain furious that iran treated u...
44894,justice yahoo settles email privacy classactio...
44895,sunnistan us and allied ‘safe zone’ plan to ta...
44896,how to blow million al jazeera america finall...


In [7]:
#Applying the function to the text column
text_clean = pd.DataFrame(dataset.text.apply(round_1))
text_clean

,text
0,washington administration has already been wi...
1,washington transgender people will be allowe...
2,washington several members of the trump campa...
3,washington trump campaign adviser george pap...
4,seattlewashington the other biggest players i...
...,...
44893,century wire says as reported earlier this w...
44894,century wire says it s a familiar theme whene...
44895,patrick henningsen century wireremember when...
44896,century wire says al jazeera america will go ...


In [8]:
#Overwriting the title and text columns
dataset['title'] = title_clean
dataset['text'] = text_clean
dataset.head()

,title,text,subject,Real/Fake
0,as us budget fight looms republicans flip thei...,washington administration has already been wi...,politicsNews,REAL
1,us military to accept transgender recruits on ...,washington transgender people will be allowe...,politicsNews,REAL
2,senior us republican senator let mr mueller do...,washington several members of the trump campa...,politicsNews,REAL
3,fbi russia probe helped by australian diplomat...,washington trump campaign adviser george pap...,politicsNews,REAL
4,trump wants postal service to charge much more...,seattlewashington the other biggest players i...,politicsNews,REAL


## Step 5: Organizing the Data 

Here I will be generating a corpus and a document-term matrix for both the fake and the real news

In [9]:
#Separating the news in to fake and real

clean_real = dataset[dataset['Real/Fake']=='REAL']
clean_fake = dataset[dataset['Real/Fake']=='FAKE']



,title,text,subject,Real/Fake
44893,mcpain john mccain furious that iran treated u...,century wire says as reported earlier this w...,Middle-east,FAKE
44894,justice yahoo settles email privacy classactio...,century wire says it s a familiar theme whene...,Middle-east,FAKE
44895,sunnistan us and allied ‘safe zone’ plan to ta...,patrick henningsen century wireremember when...,Middle-east,FAKE
44896,how to blow million al jazeera america finall...,century wire says al jazeera america will go ...,Middle-east,FAKE
44897,us navy sailors held by iranian military – si...,century wire says as predicted in its new ye...,Middle-east,FAKE


In [10]:
from sklearn.feature_extraction.text import CountVectorizer

cv=CountVectorizer(stop_words = 'english')

#DTM for real news (titles)
real_title_cv = cv.fit_transform(clean_real.title)
real_title_dtm = pd.DataFrame(real_title_cv.toarray(), columns = cv.get_feature_names())
real_title_dtm.head()

,aa,aar,aardvark,abadi,abandon,abandoned,abandoning,abandons,abbas,abbott,...,zoo,zoos,zschaepe,zuckerberg,zulia,zuma,zumas,zummar,zurich,zurichs
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
#DTM for real news (text)
real_text_cv = cv.fit_transform(clean_real.text)
real_text_dtm = pd.DataFrame(real_text_cv.toarray(), columns = cv.get_feature_names())
real_text_dtm.head()

,aa,aaa,aabo,aachen,aadhaar,aadhar,aaf,aai,aalberg,aamin,...,zwak,zweig,zweiman,zweli,zwolinski,zynga,zypries,zyries,zyuganov,émigré
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
#DTM for fake news (title)

fake_title_cv = cv.fit_transform(clean_fake.title)
fake_title_dtm = pd.DataFrame(fake_title_cv.toarray(), columns = cv.get_feature_names())
fake_title_dtm.head()

,aaron,aarp,abandon,abandoned,abandoning,abandons,abbott,abc,abcwashington,abdication,...,zombie,zombiehillary,zone,zones,zoo,zoolander,zucker,zuckerberg,état,žižek
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
#DTM for fake news (text)

fake_text_cv = cv.fit_transform(clean_fake.text)
fake_text_dtm = pd.DataFrame(fake_text_cv.toarray(), columns = cv.get_feature_names())
fake_text_dtm.head()

,aa,aaa,aaaahhhh,aaccording,aadl,aafter,aaliyah,aamer,aan,aapl,...,zurich,zusman,zvi,zvoncheck,zych,zygote,zygotes,zynga,zzzzzzzz,zzzzzzzzzzzzz
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Everything seems to be going well.I shall pickle this for the next module.

In [14]:
real_title_dtm.to_pickle('real_title_dtm.pkl')
real_text_dtm.to_pickle('real_text_dtm.pkl')
fake_title_dtm.to_pickle('fake_title_dtm.pkl')
fake_text_dtm.to_pickle('fake_text_dtm.pkl')

I shall also be pickling the cleaned data.

In [16]:
dataset.to_pickle('cleaned_dataset.pkl')

### And that's the Data Wrangling done!